# Deployment Notebook
This notebbook loads the a trained simple machine learning model from SystemLink, loads a unlabelled waveforms from SytemLink, predicts the label using the model and stores back the predictions into SystemLink Server

### Importing SystemLink Python Modules

In [1]:
from systemlink.fileingestionclient import FileIngestionClient, messages as fileingestion_messages
from systemlink.tdmsreaderclient import TDMSReaderClient, messages as tdmsreader_messages

### Importing Some Machine Learning Libraries

In [2]:
import keras
from keras.models import load_model 
from keras.utils import to_categorical

from sklearn.preprocessing import MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight

import matplotlib.pyplot as plt
from IPython.display import display, Markdown
import numpy as np
import pandas as pd

import time
import h5py
import os

Using TensorFlow backend.


### Choose the Dataset and model

In [3]:
ShiftId = "shift_0042"
wavefrom_length = 152

model_name = "simple_model"
model_version = "001"

label_to_id = {'anomalous':0, 'normal':1}
id_to_label = {0:'anomalous', 1:'normal'}

wavefrom_length = 152
no_of_classes = 2

### Fetching File IDs from SystemLink

In [4]:
fileingestionclient = FileIngestionClient(service_name='FileIngestionClient')
tdmsreaderclient = TDMSReaderClient(service_name='TDMSReaderClient')

equal_op = fileingestion_messages.QueryOperator(fileingestion_messages.QueryOperator.EQUAL)
query_shift_id = fileingestion_messages.StringQueryEntry('shiftId', ShiftId, equal_op)

file_ids = []

res = fileingestionclient.query_files(properties_query=[query_shift_id])
file_ids.extend([file.id for file in res.available_files])
no_of_waveforms = len(file_ids)
display(Markdown('<h2>Number of waveforms in the dataset : %s</h2>'%(no_of_waveforms)))

<h2>Number of waveforms in the dataset : 1000</h2>

### Loading data from SystemLink

In [5]:
X_test = np.ndarray([no_of_waveforms, wavefrom_length])
test_file_ids = []
group_name = "sensor_data"
test_idx = 0

for idx,file_id in enumerate(file_ids):
    limits = tdmsreader_messages.SkipTakeReturnCount(0, -1, True)
    channels = tdmsreaderclient.get_channels(file_id, group_name, limits)
    sensor_waveforms = []
    for channel_name in channels.names:
        channel_specifier = tdmsreader_messages.OneChannelSpecifier(
        file_id_=file_id,
        group_name_=group_name,
        channel_name_=channel_name)

        data_window = tdmsreader_messages.DataWindow(
                    xmin_=0,
                    xmax_=1000,
                    xlog_=False,
                    ymin_=0,
                    ymax_=1000,
                    ylog_=False,
                    decimatex_=0,
                    decimatey_=0,
                    show_points_=False,
                    show_lines_=False)
        xy_channels = tdmsreader_messages.XYChannels(
                    xchannel_=None,
                    ychannels_=[channel_specifier],
                    data_window_=data_window)
        channel_spec = tdmsreader_messages.ChannelSpecifications(xy_channels_=[xy_channels])
        sensor_waveform = tdmsreaderclient.query_data(channel_spec)
        sensor_waveforms.append(np.array(sensor_waveform.data[0].y[0].numeric_data))

    X_test[test_idx,:] = sensor_waveforms[0]
    test_file_ids.append(file_id)
    test_idx +=1
        
X_test_reshape = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

### Loading the model and metadata from Systemlink

In [6]:
model_filepath = "models_from_server/{}_v{}.h5".format(model_name, model_version)
train_mean_filepath ="models_from_server/{}_v{}_train_mean.npy".format(model_name, model_version)

model_query = fileingestion_messages.StringQueryEntry('Name', os.path.split(model_filepath)[-1], equal_op)
res = fileingestionclient.query_files(properties_query=[model_query])
model_file_id = res.available_files.pop().id
fileingestionclient.download_file(file_id=model_file_id, file_path=model_filepath)

train_mean_query = fileingestion_messages.StringQueryEntry('Name', os.path.split(train_mean_filepath)[-1], equal_op)
res = fileingestionclient.query_files(properties_query=[train_mean_query])
mean_file_id = res.available_files.pop().id
fileingestionclient.download_file(file_id=mean_file_id, file_path=train_mean_filepath)

model = load_model(model_filepath)
train_mean = np.load(train_mean_filepath)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


### Preprocessing the data

In [7]:
scaler = MinMaxScaler(feature_range=(-1, 1))

X_test_s = []
for x_ in X_test_reshape:
    scaler = scaler.fit(x_)
    X_test_s.append(scaler.transform(x_))
X_test_s = np.array(X_test_s)
X_test_s_mean = X_test_s - train_mean

### Predicting and storing the predicted labels in SystemLink as metadata

In [8]:
preds_prob = model.predict(X_test_s_mean[:,:,0], verbose=1, batch_size=64)
preds = np.argmax(preds_prob, axis=1)

for file_id, prediction, prob in zip(file_ids, preds,preds_prob):
    fileingestionclient.update_file_metadata(file_id=file_id, replace_existing=False, properties={"predictedLabel":id_to_label[prediction],
                                                                                                 "confidence":"%0.2f%%"%(max(prob)*100)})

os.remove(model_filepath)
os.remove(train_mean_filepath)

display(Markdown('<h2>Total Number of Waveforms : %s</h2>'%(len(preds))))
display(Markdown('<h4>Predicted as Normal : %s</h4>'%(len(preds[preds == 1]))))
display(Markdown('<h4>Predicted as Anomalous : %s</h4><hr>'%(len(preds[preds == 0]))))

1000/1000 [==============================] - 0s 119us/step


<h2>Total Number of Waveforms : 1000</h2>

<h4>Predicted as Normal : 840</h4>

<h4>Predicted as Anomalous : 160</h4><hr>